In [2]:
!uv add matplotlib
!uv add kiwipiepy

Resolved 286 packages in 1ms
Audited 261 packages in 0.14ms
Resolved 286 packages in 1ms
Audited 261 packages in 0.11ms


In [95]:
import pandas as pd
import matplotlib.pyplot as plt

In [96]:
file1 = r'C:\Users\user\Desktop\woogawooga\woogawooga_project\dataset\phishing_data.csv'

In [97]:
file2 = r'C:\Users\user\Desktop\woogawooga\woogawooga_project\dataset\일반통화대화통합.csv'
df2 = pd.read_csv(file2)

In [98]:
df1 = pd.read_csv(file1)

In [99]:
df1.head()

,file_name,phishing_type,speaker,text
0,phishing_001,가족지인사칭형,1,아빠? 목소리가 좀 이상한데요.
1,phishing_001,가족지인사칭형,1,무슨 일인데 그렇게 급해요?
2,phishing_001,가족지인사칭형,0,아빠 핸드폰이 고장 나서 친구 폰으로 걸었어. 급하게 돈이 필요해서.
3,phishing_001,가족지인사칭형,0,교통사고가 나서 병원비가 필요해. 믿고 바로 보내줄 수 있지?
4,phishing_001,가족지인사칭형,0,"여보세요, OOO야. 나 아빠야."


In [ ]:
# 피싱 타입별 텍스트 총 글자수 집계 및 비율 추가, 내림차순 정렬
text_length_sum = (
    df1.groupby('phishing_type')['text']
       .apply(lambda x: x.str.len().sum())
       .reset_index(name='total_text_length')
)
# 비율 컬럼 추가 (소수점 첫째 자리까지)
text_length_sum['percent'] = (
    text_length_sum['total_text_length'] / text_length_sum['total_text_length'].sum() * 100
).round(1)
# 내림차순 정렬
text_length_sum = text_length_sum.sort_values(by='total_text_length', ascending=False).reset_index(drop=True)
print(text_length_sum)


In [ ]:
# 파일 네임별, 피싱 타입별로 중복 제거
df_unique = df1.drop_duplicates(subset=['file_name', 'phishing_type'])

# 피싱 타입별로 파일 네임 개수 집계
count = df_unique['phishing_type'].value_counts()
percent = df_unique['phishing_type'].value_counts(normalize=True).mul(100).round(1)

# count와 percent를 합쳐서 데이터프레임으로 정리
result = pd.DataFrame({'count': count, 'percent': percent})
result = result.reset_index().rename(columns={'index': 'phishing_type'})
print(result)

In [ ]:
# 비율 데이터 바로 추출
phishing_counts = df1['phishing_type'].value_counts(normalize=True) * 100

# 원형 그래프 그리기
plt.figure(figsize=(8, 8))
plt.pie(
    phishing_counts, 
    labels=phishing_counts.index, 
    autopct='%1.1f%%', 
    startangle=140, 
    textprops={'fontsize': 12}
)
plt.title('보이스피싱 유형별 비율', fontsize=16)
plt.axis('equal')  # 원형 유지
plt.show()

In [ ]:
import pandas as pd
from collections import Counter
import re

# 한글 단어만 추출하는 함수
def extract_keywords(text):
    # 한글 단어만 추출 (띄어쓰기 기준)
    keywords = re.findall(r'\b[가-힣]+\b', text)
    return keywords

# 키워드 빈도수 계산 함수
def keyword_frequency(df, text_column):
    all_keywords = []
    for text in df[text_column]:
        if pd.isnull(text):  # 결측치 방지
            continue
        all_keywords.extend(extract_keywords(str(text)))
    keyword_counts = Counter(all_keywords)
    keyword_freq_df = pd.DataFrame(keyword_counts.items(), columns=['keyword', 'frequency'])
    keyword_freq_df = keyword_freq_df.sort_values(by='frequency', ascending=False).reset_index(drop=True)
    return keyword_freq_df

# 사용 예시
# keyword_freq_df = keyword_frequency(df1, 'text')
# display(keyword_freq_df.head())


In [ ]:
# 사용
keyword_freq_df = keyword_frequency(df1, 'text')

In [ ]:
keyword_freq_df["keyword"].nunique()

In [ ]:
keyword_freq_df.head(100)

In [100]:
import matplotlib.pyplot as plt

plt.rc('font', family='Malgun Gothic')  # 윈도우에서 기본 제공
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
import pandas as pd
from collections import Counter
import re
import matplotlib.pyplot as plt

# 한글 단어만 추출하는 함수
def extract_keywords(text):
    keywords = re.findall(r'\b[가-힣]+\b', text)
    return keywords

# 키워드 빈도수 계산 함수
def keyword_frequency(df, text_column):
    all_keywords = []
    for text in df[text_column]:
        if pd.isnull(text):
            continue
        all_keywords.extend(extract_keywords(str(text)))
    keyword_counts = Counter(all_keywords)
    keyword_freq_df = pd.DataFrame(keyword_counts.items(), columns=['keyword', 'frequency'])
    keyword_freq_df = keyword_freq_df.sort_values(by='frequency', ascending=False).reset_index(drop=True)
    return keyword_freq_df

# 예시: 데이터프레임(df1)에서 상위 20개 추출
keyword_freq_df = keyword_frequency(df1, 'text')
top_100 = keyword_freq_df.head(20)

# 원형 차트 그리기
plt.figure(figsize=(10, 8))
plt.pie(top_100['frequency'], labels=top_100['keyword'], autopct='%1.1f%%', startangle=140)
plt.title('상위 20개 키워드 빈도수 원형 차트')
plt.axis('equal')
plt.show()


In [101]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44519 entries, 0 to 44518
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   file_name      44519 non-null  object
 1   phishing_type  44519 non-null  object
 2   speaker        44519 non-null  int64 
 3   text           44519 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.4+ MB


In [102]:
df2.head()

,file_name,category,subcategory,speaker,text
0,Empathy_기쁨_부모자녀_조손_127.json,기쁨,부모자녀/조손,1,"엄마, 이제 여름이니까 나 코토리베이지라는 색으로 염색했어."
1,Empathy_기쁨_부모자녀_조손_127.json,기쁨,부모자녀/조손,2,우리 딸 상상만 해도 너무 예쁠 것 같은데?
2,Empathy_기쁨_부모자녀_조손_127.json,기쁨,부모자녀/조손,1,엄마도 오글거리는 말 진짜 잘해. 근데 내가 생각해도 너무 잘 어울리는 것 같아. ...
3,Empathy_기쁨_부모자녀_조손_127.json,기쁨,부모자녀/조손,2,진짜? 그 정도로 좋아하는 거 보니 우리 딸이 엄청 마음에 들었나 보네?
4,Empathy_기쁨_부모자녀_조손_127.json,기쁨,부모자녀/조손,1,맞아. 이번에는 완전 찰떡이야. 계속 거울을 들여다보게 돼.


In [22]:
df1.head()

,file_name,phishing_type,speaker,text
0,phishing_001,가족지인사칭형,1,아빠? 목소리가 좀 이상한데요.
1,phishing_001,가족지인사칭형,1,무슨 일인데 그렇게 급해요?
2,phishing_001,가족지인사칭형,0,아빠 핸드폰이 고장 나서 친구 폰으로 걸었어. 급하게 돈이 필요해서.
3,phishing_001,가족지인사칭형,0,교통사고가 나서 병원비가 필요해. 믿고 바로 보내줄 수 있지?
4,phishing_001,가족지인사칭형,0,"여보세요, OOO야. 나 아빠야."


In [17]:
from konlpy.tag import Kkma

kkma = Kkma()

def kkma_tokenizer(text):
    # 형태소 및 품사 태깅
    pos = kkma.pos(text)
    # 주요 품사(명사, 동사, 형용사, 감탄사)만 추출
    keywords = [word for word, tag in pos if tag in ['NNG', 'NNP', 'VV', 'VA']]
    return keywords

# 파일에서 텍스트 읽기
file_path = r'C:\Users\user\Desktop\woogawooga\woogawooga_project\dataset\2만자.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    sentence = file.read()

print(kkma_tokenizer(sentence))


['인간과', '인공지능', '공존', '위하', '길', '여정', '인공지능', '이상', '공상', '과학', '소설', '속', '상상', '일상', '속', '들어오', '이메일', '자동', '완성', '음성', '비서', '금융', '사기', '탐지', '자율', '주행', '영향력', '커지', '그러', '기술', '발전', '속도', '인간', '사회적', '윤리적', '수용', '속도', '앞지르', '공존', '키워드', '대하', '고민', '하', '시점', '이', '르', '인간과', '인공지능', '관계', '단순', '도구적', '상호', '작용', '넘', '서로', '이해', '책임', '나누', '동반자', '개념', '확장', '기술', '역사', '속', '인간', '새롭', '도구', '개발', '바퀴', '증기', '기관', '컴퓨터', '지금', '인공지능', '그러', '인공지능', '기존', '기술', '차원', '다르', '존재', '데이터', '학습', '의사', '결정', '하', '인간', '낫', '판단', '내리', '하', '특성', '인공지능', '단순', '도', '로만', '간주', '어렵', '만들', '인공지능', '삶', '편리', '만들', '동시', '윤리', '고용', '교육', '정체성', '다양', '분야', '도전', '과제', '던지', '우선', '고용', '문제', '살펴보', '자동화', '인공지능', '확산', '일부', '직업', '소멸', '초래', '있', '반복', '작업', '규칙', '기반', '업무', '빠르', '대체', '있', '그러', '과정', '일', '자리', '감소', '가', '아니', '일의', '성격', '변화', '의미', '새롭', '직업', '군', '생겨나', '인간', '창의', '감', '성적', '능력', '요구', '받', '직무', '집중', '되', '전환', '부드럽', '이루', '위하', '교육', '시스템', '

In [29]:
from konlpy.tag import Kkma

kkma = Kkma()

def kkma_tokenizer_with_stem(text):
    pos = kkma.pos(text)
    keywords = []
    for word, tag in pos:
        # 명사, 고유명사는 그대로 사용
        if tag in ['NNG', 'NNP']:
            keywords.append(word)
        # 동사, 형용사는 기본형(어간)으로 변환
        elif tag in ['VV', 'VA']:
            # Kkma는 기본적으로 어간을 반환하므로 그대로 사용
            keywords.append(word)
    return keywords

# 파일에서 텍스트 읽기
file_path = r'C:\Users\user\Desktop\woogawooga\woogawooga_project\dataset\2만자.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    sentence = file.read()

print(kkma_tokenizer_with_stem(sentence))

['인간과', '인공지능', '공존', '위하', '길', '여정', '인공지능', '이상', '공상', '과학', '소설', '속', '상상', '일상', '속', '들어오', '이메일', '자동', '완성', '음성', '비서', '금융', '사기', '탐지', '자율', '주행', '영향력', '커지', '그러', '기술', '발전', '속도', '인간', '사회적', '윤리적', '수용', '속도', '앞지르', '공존', '키워드', '대하', '고민', '하', '시점', '이', '르', '인간과', '인공지능', '관계', '단순', '도구적', '상호', '작용', '넘', '서로', '이해', '책임', '나누', '동반자', '개념', '확장', '기술', '역사', '속', '인간', '새롭', '도구', '개발', '바퀴', '증기', '기관', '컴퓨터', '지금', '인공지능', '그러', '인공지능', '기존', '기술', '차원', '다르', '존재', '데이터', '학습', '의사', '결정', '하', '인간', '낫', '판단', '내리', '하', '특성', '인공지능', '단순', '도', '로만', '간주', '어렵', '만들', '인공지능', '삶', '편리', '만들', '동시', '윤리', '고용', '교육', '정체성', '다양', '분야', '도전', '과제', '던지', '우선', '고용', '문제', '살펴보', '자동화', '인공지능', '확산', '일부', '직업', '소멸', '초래', '있', '반복', '작업', '규칙', '기반', '업무', '빠르', '대체', '있', '그러', '과정', '일', '자리', '감소', '가', '아니', '일의', '성격', '변화', '의미', '새롭', '직업', '군', '생겨나', '인간', '창의', '감', '성적', '능력', '요구', '받', '직무', '집중', '되', '전환', '부드럽', '이루', '위하', '교육', '시스템', '

In [35]:
from konlpy.tag import Kkma

kkma = Kkma()

def kkma_tokenizer_with_lemma(text):
    pos = kkma.pos(text)
    keywords = []
    for word, tag in pos:
        # 명사, 고유명사: 그대로 사용
        if tag in ['NNG', 'NNP']:
            keywords.append(word)
        # 동사, 형용사: 기본형(표제어)로 변환
        elif tag in ['VV', 'VA']:
            # Kkma는 기본적으로 어간(표제어)을 반환
            keywords.append(word)
    return keywords

# 파일에서 텍스트 읽기
file_path = r'C:\Users\user\Desktop\woogawooga\woogawooga_project\dataset\2만자.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    sentence = file.read()

print(kkma_tokenizer_with_lemma(sentence))

['인간과', '인공지능', '공존', '위하', '길', '여정', '인공지능', '이상', '공상', '과학', '소설', '속', '상상', '일상', '속', '들어오', '이메일', '자동', '완성', '음성', '비서', '금융', '사기', '탐지', '자율', '주행', '영향력', '커지', '그러', '기술', '발전', '속도', '인간', '사회적', '윤리적', '수용', '속도', '앞지르', '공존', '키워드', '대하', '고민', '하', '시점', '이', '르', '인간과', '인공지능', '관계', '단순', '도구적', '상호', '작용', '넘', '서로', '이해', '책임', '나누', '동반자', '개념', '확장', '기술', '역사', '속', '인간', '새롭', '도구', '개발', '바퀴', '증기', '기관', '컴퓨터', '지금', '인공지능', '그러', '인공지능', '기존', '기술', '차원', '다르', '존재', '데이터', '학습', '의사', '결정', '하', '인간', '낫', '판단', '내리', '하', '특성', '인공지능', '단순', '도', '로만', '간주', '어렵', '만들', '인공지능', '삶', '편리', '만들', '동시', '윤리', '고용', '교육', '정체성', '다양', '분야', '도전', '과제', '던지', '우선', '고용', '문제', '살펴보', '자동화', '인공지능', '확산', '일부', '직업', '소멸', '초래', '있', '반복', '작업', '규칙', '기반', '업무', '빠르', '대체', '있', '그러', '과정', '일', '자리', '감소', '가', '아니', '일의', '성격', '변화', '의미', '새롭', '직업', '군', '생겨나', '인간', '창의', '감', '성적', '능력', '요구', '받', '직무', '집중', '되', '전환', '부드럽', '이루', '위하', '교육', '시스템', '

In [18]:
from konlpy.tag import Komoran

komoran = Komoran()

def komoran_tokenizer(text):
    pos = komoran.pos(text)
    keywords = [word for word, tag in pos if tag in ['NNG', 'NNP', 'VV', 'VA']]
    return keywords

# 파일에서 텍스트 읽기
file_path = r'C:\Users\user\Desktop\woogawooga\woogawooga_project\dataset\2만자.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    sentence = file.read()

print(komoran_tokenizer(sentence))

['인간', '인공지능', '공존', '위하', '길', '여정', '인공지능', '이상', '공상', '과학', '소설', '속', '상상', '일상', '속', '들어오', '있', '메일', '자동', '완성', '음성', '비서', '금융', '사기', '탐지', '자율', '주행', '영향력', '커지', '기술', '발전', '속도', '인간', '사회', '윤리', '수용', '속도', '앞지르', '공존', '키워드', '대하', '고민', '하', '시점', '이르', '인간', '인공지능', '관계', '도구', '상호작용', '넘', '서로', '이해', '책임', '나누', '동반자', '개념', '확장', '기술의 역사', '속', '인간', '새롭', '도구', '개발', '바퀴', '증기', '기관', '컴퓨터', '지금', '인공지능', '인공지능', '기존', '기술', '차원', '존재', '데이터', '학습', '의사', '결정', '하', '때', '인간', '판단', '내리', '특성', '인공지능', '도구', '로만', '간주', '어렵', '만들', '인공지능', '삶', '편리', '만들', '동시', '윤리', '고용', '교육', '정체성', '분야', '도전', '과제', '던지', '고용', '문제', '살펴보', '자동화', '인공지능', '확산', '일부', '직업', '소멸', '초래', '반복', '작업', '규칙', '기반', '업무', '빠르', '대체', '과정', '일자리', '감소', '일', '성격', '변화', '의미', '새롭', '직업', '군', '생겨나', '인간', '창의', '감성', '능력', '요구', '받', '직무', '집중', '되', '전환', '부드럽', '이루어지', '위하', '교육', '시스템', '개편', '평생', '학습', '기반', '강화', '필요', '기술', '교육', '넘', '인간', '고유', '역량', '비판적 사고', '창의성', '공감',

In [24]:
from konlpy.tag import Komoran

komoran = Komoran()

def komoran_tokenizer(text):
    # stem=True로 설정하면 동사/형용사가 기본형(어간)으로 반환됨
    pos = komoran.pos(text, stem=True)
    keywords = [word for word, tag in pos if tag in ['NNG', 'NNP', 'VV', 'VA']]
    return keywords

# 파일에서 텍스트 읽기
file_path = r'C:\Users\user\Desktop\woogawooga\woogawooga_project\dataset\2만자.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    sentence = file.read()

print(komoran_tokenizer(sentence))

TypeError: Komoran.pos() got an unexpected keyword argument 'stem'

In [19]:
from kiwipiepy import Kiwi

kiwi = Kiwi()

def kiwi_tokenizer(text):
    tokens = kiwi.tokenize(text)
    keywords = [token.form for token in tokens if token.tag in ['NNG', 'NNP', 'VV', 'VA']]
    return keywords

# 파일에서 텍스트 읽기
file_path = r'C:\Users\user\Desktop\woogawooga\woogawooga_project\dataset\2만자.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    sentence = file.read()

print(kiwi_tokenizer(sentence))


['인간', '인공', '지능', '공존', '위하', '길', '여정', '인공지능', '이상', '공상', '과학', '소설', '속', '상상', '일상', '속', '들어오', '이메일', '자동', '완성', '음성', '비서', '금융', '사기', '탐지', '자율', '주행', '영향력', '커지', '기술', '발전', '속도', '인간', '사회', '윤리', '수용', '속도', '앞지르', '공존', '키워드', '대하', '고민', '시점', '인간', '인공지능', '관계', '단순', '도구', '상호', '작용', '넘', '서로', '이해', '책임', '나누', '동반자', '개념', '확장', '기술', '역사', '속', '인간', '도구', '개발', '바퀴', '증기', '기관', '컴퓨터', '지금', '인공지능', '인공지능', '기존', '기술', '차원', '다르', '존재', '데이터', '학습', '의사', '결정', '하', '인간', '판단', '내리', '특성', '인공지능', '단순', '도구', '간주', '인공지능', '삶', '편리', '동시', '윤리', '고용', '교육', '정체성', '다양', '분야', '도전', '과제', '던지', '고용', '문제', '살펴보', '자동화', '인공지능', '확산', '일부', '직업', '소멸', '초래', '있', '반복', '작업', '규칙', '기반', '업무', '빠르', '대체', '있', '과정', '일자리', '감소', '일', '성격', '변화', '의미', '직업군', '생겨나', '인간', '창의', '감성', '능력', '요구', '직무', '집중', '되', '전환', '이루', '위하', '교육', '시스템', '개편', '평생', '학습', '기반', '강화', '필요', '단순', '기술', '교육', '넘', '인간', '고유', '역량', '비판', '사고', '창의성', '공감', '능력', '기르', '방향', '전환

In [31]:
from kiwipiepy import Kiwi

kiwi = Kiwi()

def kiwi_tokenizer(text):
    tokens = kiwi.tokenize(text)
    keywords = []
    for token in tokens:
        if token.tag in ['NNG', 'NNP']:
            keywords.append(token.form)
        elif token.tag in ['VV', 'VA']:
            keywords.append(token.lemma)  # 기본형(어간) 추출
    return keywords

# 파일에서 텍스트 읽기
file_path = r'C:\Users\user\Desktop\woogawooga\woogawooga_project\dataset\2만자.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    sentence = file.read()

print(kiwi_tokenizer(sentence))


['인간', '인공', '지능', '공존', '위하다', '길다', '여정', '인공지능', '이상', '공상', '과학', '소설', '속', '상상', '일상', '속', '들어오다', '이메일', '자동', '완성', '음성', '비서', '금융', '사기', '탐지', '자율', '주행', '영향력', '커지다', '기술', '발전', '속도', '인간', '사회', '윤리', '수용', '속도', '앞지르다', '공존', '키워드', '대하다', '고민', '시점', '인간', '인공지능', '관계', '단순', '도구', '상호', '작용', '넘다', '서로', '이해', '책임', '나누다', '동반자', '개념', '확장', '기술', '역사', '속', '인간', '도구', '개발', '바퀴', '증기', '기관', '컴퓨터', '지금', '인공지능', '인공지능', '기존', '기술', '차원', '다르다', '존재', '데이터', '학습', '의사', '결정', '하다', '인간', '판단', '내리다', '특성', '인공지능', '단순', '도구', '간주', '인공지능', '삶', '편리', '동시', '윤리', '고용', '교육', '정체성', '다양', '분야', '도전', '과제', '던지다', '고용', '문제', '살펴보다', '자동화', '인공지능', '확산', '일부', '직업', '소멸', '초래', '있다', '반복', '작업', '규칙', '기반', '업무', '빠르다', '대체', '있다', '과정', '일자리', '감소', '일', '성격', '변화', '의미', '직업군', '생겨나다', '인간', '창의', '감성', '능력', '요구', '직무', '집중', '되다', '전환', '이루다', '위하다', '교육', '시스템', '개편', '평생', '학습', '기반', '강화', '필요', '단순', '기술', '교육', '넘다', '인간', '고유', '역량', '비판', '사고', '창의성', '공감', 

In [34]:
from kiwipiepy import Kiwi

kiwi = Kiwi()

def kiwi_tokenizer_with_lemma(text):
    tokens = kiwi.tokenize(text)
    keywords = []
    for token in tokens:
        # 명사, 고유명사: 원형 그대로 사용
        if token.tag in ['NNG', 'NNP']:
            keywords.append(token.form)
        # 동사, 형용사: 표제어(기본형)로 변환
        elif token.tag in ['VV', 'VA']:
            keywords.append(token.lemma)
    return keywords

# 파일에서 텍스트 읽기
file_path = r'C:\Users\user\Desktop\woogawooga\woogawooga_project\dataset\2만자.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    sentence = file.read()

print(kiwi_tokenizer_with_lemma(sentence))

['인간', '인공', '지능', '공존', '위하다', '길다', '여정', '인공지능', '이상', '공상', '과학', '소설', '속', '상상', '일상', '속', '들어오다', '이메일', '자동', '완성', '음성', '비서', '금융', '사기', '탐지', '자율', '주행', '영향력', '커지다', '기술', '발전', '속도', '인간', '사회', '윤리', '수용', '속도', '앞지르다', '공존', '키워드', '대하다', '고민', '시점', '인간', '인공지능', '관계', '단순', '도구', '상호', '작용', '넘다', '서로', '이해', '책임', '나누다', '동반자', '개념', '확장', '기술', '역사', '속', '인간', '도구', '개발', '바퀴', '증기', '기관', '컴퓨터', '지금', '인공지능', '인공지능', '기존', '기술', '차원', '다르다', '존재', '데이터', '학습', '의사', '결정', '하다', '인간', '판단', '내리다', '특성', '인공지능', '단순', '도구', '간주', '인공지능', '삶', '편리', '동시', '윤리', '고용', '교육', '정체성', '다양', '분야', '도전', '과제', '던지다', '고용', '문제', '살펴보다', '자동화', '인공지능', '확산', '일부', '직업', '소멸', '초래', '있다', '반복', '작업', '규칙', '기반', '업무', '빠르다', '대체', '있다', '과정', '일자리', '감소', '일', '성격', '변화', '의미', '직업군', '생겨나다', '인간', '창의', '감성', '능력', '요구', '직무', '집중', '되다', '전환', '이루다', '위하다', '교육', '시스템', '개편', '평생', '학습', '기반', '강화', '필요', '단순', '기술', '교육', '넘다', '인간', '고유', '역량', '비판', '사고', '창의성', '공감', 

In [30]:
from konlpy.tag import Okt

okt = Okt()

def okt_tokenizer(text):
    # 형태소 분석 후 원하는 품사 필터링
    tokens = okt.pos(text, stem=True)  # stem=True는 동사 원형 복원
    keywords = [word for word, tag in tokens if tag in ['Noun', 'Verb', 'Adjective', 'Exclamation']]
    return keywords

# 파일에서 텍스트 읽기
file_path = r'C:\Users\user\Desktop\woogawooga\woogawooga_project\dataset\2만자.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    sentence = file.read()

print(okt_tokenizer(sentence))

['인간', '인공', '지능', '공존', '위', '기다', '여정', '인공', '지능', '더', '이상', '공상', '과학', '소설', '속', '상상', '아니다', '우리', '일상', '속', '깊이', '들어오다', '있다', '이메일', '자동', '완성', '음성', '비서', '금융', '사기', '탐지', '자율', '주행', '그', '영향력', '날로', '커지다', '있다', '기술', '발전', '속도', '인간', '사회', '윤리', '수용', '속도', '앞', '지르다', '우리', '공존', '키', '워드', '대해', '진지하다', '고민', '하다', '하다', '시점', '이르렀다', '인간', '인공', '지능', '관계', '단순하다', '도구', '상호작용', '넘다', '서로', '이해', '책임', '나누다', '동반', '자의', '개념', '확장', '되다', '있다', '기술', '역사', '속', '인간', '늘', '새롭다', '도구', '개발', '해오다', '바퀴', '증기기관', '컴퓨터', '지금', '인공', '지능', '인공', '지능', '기존', '기술', '차원', '다른', '존재', '그것', '데이터', '학습', '의사결정', '하다', '때로는', '인간', '더', '나은', '판단', '리기', '하다', '이렇다', '특성', '인공', '지능', '단순하다', '도구', '로만', '간주', '하다', '어렵다', '만들다', '인공', '지능', '우리', '삶', '편리하다', '만들다', '동시', '윤리', '고용', '교육', '정체', '등', '다양하다', '분야', '도전', '과제', '던지다', '우선', '고용', '문제', '살펴보다', '자동화', '인공', '지능', '확산', '일부', '직업', '소멸', '초래', '하다', '수', '있다', '반복', '작업', '규칙', '기반', '업무', '가다', '빠르다', '대체', 

In [103]:
from konlpy.tag import Komoran

# Komoran 객체 생성
komoran = Komoran()

# 추출할 품사 태그
target_tags = ['NNG', 'NNP', 'VV', 'VA']

def komoran_tokenizer(text):
    pos = komoran.pos(text)
    keywords = [word for word, tag in pos if tag in target_tags]
    # # "네", "예"가 원문에 있고 결과에 없으면 추가
    # for ex in ['네', '예']:
    #     if ex in text and ex not in keywords:
    #         keywords.append(ex)
    # "OOO"가 있으면 제거
    keywords = [word for word in keywords if word != 'OOO']
    # 빈 리스트면 '없음'으로 반환
    return keywords if keywords else ['없음']

# 데이터프레임 전체에 적용 (새 컬럼 생성 또는 덮어쓰기)
df1['komoran_tokens'] = df1['text'].apply(komoran_tokenizer)

In [104]:
from konlpy.tag import Komoran

# Komoran 객체 생성
komoran = Komoran()

# 추출할 품사 태그
target_tags = ['NNG', 'NNP', 'VV', 'VA']

def komoran_tokenizer(text):
    pos = komoran.pos(text)
    keywords = [word for word, tag in pos if tag in target_tags]
    # # "네", "예"가 원문에 있고 결과에 없으면 추가
    # for ex in ['네', '예']:
    #     if ex in text and ex not in keywords:
    #         keywords.append(ex)
    # "OOO"가 있으면 제거
    keywords = [word for word in keywords if word != 'OOO']
    # 빈 리스트면 '없음'으로 반환
    return keywords if keywords else ['없음']

# 데이터프레임 전체에 적용 (새 컬럼 생성 또는 덮어쓰기)
df2['komoran_tokens'] = df2['text'].apply(komoran_tokenizer)

In [105]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336813 entries, 0 to 336812
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   file_name       336813 non-null  object
 1   category        336813 non-null  object
 2   subcategory     336813 non-null  object
 3   speaker         336813 non-null  int64 
 4   text            336813 non-null  object
 5   komoran_tokens  336813 non-null  object
dtypes: int64(1), object(5)
memory usage: 15.4+ MB


In [86]:
df2.head(10)

,file_name,category,subcategory,speaker,text,komoran_tokens
0,Empathy_기쁨_부모자녀_조손_127.json,기쁨,부모자녀/조손,1,"엄마, 이제 여름이니까 나 코토리베이지라는 색으로 염색했어.","[엄마, 여름, 코, 토리, 베이지, 색, 염색]"
1,Empathy_기쁨_부모자녀_조손_127.json,기쁨,부모자녀/조손,2,우리 딸 상상만 해도 너무 예쁠 것 같은데?,"[딸, 상상, 하, 예쁘, 같, 예]"
2,Empathy_기쁨_부모자녀_조손_127.json,기쁨,부모자녀/조손,1,엄마도 오글거리는 말 진짜 잘해. 근데 내가 생각해도 너무 잘 어울리는 것 같아. ...,"[엄마, 오, 글, 거리, 말, 생각, 어울리, 같, 친구, 자랑]"
3,Empathy_기쁨_부모자녀_조손_127.json,기쁨,부모자녀/조손,2,진짜? 그 정도로 좋아하는 거 보니 우리 딸이 엄청 마음에 들었나 보네?,"[진짜, 정도, 좋아하, 보니, 딸, 마음, 들, 네]"
4,Empathy_기쁨_부모자녀_조손_127.json,기쁨,부모자녀/조손,1,맞아. 이번에는 완전 찰떡이야. 계속 거울을 들여다보게 돼.,"[맞, 이번, 완전, 찰떡, 거울, 들여다보, 되]"
5,Empathy_기쁨_부모자녀_조손_127.json,기쁨,부모자녀/조손,2,엄마도 염색하고 나서 마음에 들면 하늘을 날고 있는 것만 같아. 우리 딸 정말 기분...,"[엄마, 염색, 나서, 마음, 들, 하늘, 날, 있, 같, 딸, 기분, 좋, 네]"
6,Empathy_기쁨_부모자녀_조손_127.json,기쁨,부모자녀/조손,1,"응, 기분 전환 완벽하게 됐어. 여름 동안에는 계속 이 머리 색을 유지하고 싶어. ...","[응, 기분, 전환, 완벽, 되, 여름, 동안, 머리, 색, 유지, 사진, 찍]"
7,Empathy_기쁨_부모자녀_조손_127.json,기쁨,부모자녀/조손,2,우리 딸은 꾸미는 거 잘하니까 예쁘게 찍어서 엄마한테도 사진 보내줄래?,"[딸, 꾸미, 예쁘, 찍, 엄마, 사진, 보내, 예]"
8,Empathy_기쁨_부모자녀_조손_127.json,기쁨,부모자녀/조손,1,"알겠어, 엄마한테 가장 먼저 보내줄 거야. 내 머리 색 보고 너무 잘 어울려서 깜짝...","[알, 엄마, 보내, 머리, 색, 보, 어울리, 놀라]"
9,Empathy_기쁨_부모자녀_조손_127.json,기쁨,부모자녀/조손,2,"참나, 얼마나 예쁜지 엄마가 두 눈 똑바로 뜨고 지켜볼 거야.","[참, 예쁘, 엄마, 눈, 뜨, 지켜보, 예]"


In [87]:
df2.tail(10)

,file_name,category,subcategory,speaker,text,komoran_tokens
336803,S038329.json,전자상거래,"결제, 취소, 환불 문의",0,네네. 그러면 고거 취소하고 다시 장바구니에 좀 담아주실 수 있으시겠어요?,"[네네, 고거, 취소, 장바구니, 담, 네]"
336804,S038329.json,전자상거래,"결제, 취소, 환불 문의",1,o/ 그러시면은 기존 결제하셨던 교재 전부 취소하시고 (1번)/(한 번)에 다시 결...,"[그렇, 기존, 결제, 교재, 취소, 결제, 무료, 배, 송, 같, 이렇, 진행, ..."
336805,S038329.json,전자상거래,"결제, 취소, 환불 문의",0,오늘 주문한 거에 한꺼번에 이렇게 하는 방법은 없습니까?,"[오늘, 주문, 이렇, 하, 방법, 없]"
336806,S038329.json,전자상거래,"결제, 취소, 환불 문의",0,예. 그런데 지금 이제 저희가 추가하는 거는 (1권)/(한 권)이어서.,"[예, 추가, 권]"
336807,S038329.json,전자상거래,"결제, 취소, 환불 문의",1,n/ 저희 (50000원)/(오만 원) 이상 결제하시면 배송비는 따로 부과가 안 되세요.,"[오만, 이상, 결제, 배, 송, 비, 부과, 되]"
336808,S038329.json,전자상거래,"결제, 취소, 환불 문의",0,n/ 네네 통합 주문 방식으로 하면 배송료가 안 드나요?,"[네네, 통합, 주문, 방식, 하, 배, 송료, 들, 네]"
336809,S038329.json,전자상거래,"결제, 취소, 환불 문의",0,o/ 네 저희 오늘 교재를 샀는데요. 그 네 빠진 교재가 있어서 추가 주문하고 싶은데.,"[오늘, 교재, 사, 빠지, 교재, 있, 추가, 주문, 네]"
336810,S038329.json,전자상거래,"결제, 취소, 환불 문의",0,캐시로 넣어주시면 그걸로 그냥 결제하면 되죠?,"[캐시, 넣, 결제, 되]"
336811,S038329.json,전자상거래,"결제, 취소, 환불 문의",1,제가 이거 (2개)/(두 개)만 좀 취소해드릴 테니까 이 (2개)/(두 개)랑 다시...,"[제가, 취소, 테, 결제, 만원, 오만, 넘, 같, 이렇, 해보]"
336812,S038329.json,전자상거래,"결제, 취소, 환불 문의",1,네. 여기다 그냥 추가로 결제하셔서 그냥 (추) 차액만 결제하시면 되거든요?,"[네, 여기, 추가, 결제, 추, 차액, 결제, 되]"


In [107]:
import pandas as pd
from collections import Counter

def get_keyword_freq_filtered_with_ratio(df, column_name='komoran_tokens', min_freq=10):
    # 모든 키워드 리스트를 하나로 합침
    all_keywords = [keyword for sublist in df[column_name] for keyword in sublist]
    # 키워드 빈도수 계산
    keyword_counts = Counter(all_keywords)
    # 데이터프레임으로 변환
    keyword_freq_df = pd.DataFrame(keyword_counts.items(), columns=['keyword', 'frequency'])
    # 전체 키워드 수
    total_keywords = sum(keyword_counts.values())
    # 비율 컬럼 추가
    keyword_freq_df['ratio'] = keyword_freq_df['frequency'] / total_keywords * 100
    # min_freq 미만 제거 및 내림차순 정렬
    keyword_freq_df_filtered = keyword_freq_df[keyword_freq_df['frequency'] >= min_freq] \
                                .sort_values(by='frequency', ascending=False) \
                                .reset_index(drop=True)
    return keyword_freq_df_filtered

# 사용 예시
filtered_df = get_keyword_freq_filtered_with_ratio(df1)
print(filtered_df.head())


  keyword  frequency     ratio
0       하       9760  3.500341
1       되       8942  3.206972
2      본인       5786  2.075100
3      없음       4249  1.523868
4       있       4181  1.499480


In [113]:
import pandas as pd
from collections import Counter

def get_keyword_freq_filtered_with_ratio(df, column_name='komoran_tokens', min_freq=10):
    # 모든 키워드 리스트를 하나로 합침
    all_keywords = [keyword for sublist in df[column_name] for keyword in sublist]
    # 키워드 빈도수 계산
    keyword_counts = Counter(all_keywords)
    # 데이터프레임으로 변환
    keyword_freq_df = pd.DataFrame(keyword_counts.items(), columns=['keyword', 'frequency'])
    # 전체 키워드 수
    total_keywords = sum(keyword_counts.values())
    # 비율 컬럼 추가
    keyword_freq_df['ratio'] = keyword_freq_df['frequency'] / total_keywords * 100
    # min_freq 미만 제거 및 내림차순 정렬
    keyword_freq_df_filtered = keyword_freq_df[keyword_freq_df['frequency'] >= min_freq] \
                                .sort_values(by='frequency', ascending=False) \
                                .reset_index(drop=True)
    return keyword_freq_df_filtered

# 사용 예시
filtered_df = get_keyword_freq_filtered_with_ratio(df2)
print(filtered_df.head())


  keyword  frequency     ratio
0       하      67207  3.985840
1       되      57743  3.424559
2       있      30882  1.831516
3      없음      28574  1.694636
4      확인      28223  1.673819


In [89]:
filtered_df.head(20)

,keyword,frequency
0,네,89216
1,하,67212
2,되,57743
3,아,44156
4,있,30882
5,확인,28223
6,예,28209
7,어,27265
8,그렇,22924
9,같,19355


In [90]:
filtered_df = filtered_df[filtered_df['frequency'] >= 10].reset_index(drop=True)

In [91]:
filtered_df.nunique()

keyword      6178
frequency     830
dtype: int64

In [114]:
import pandas as pd

# 예시: filtered_df가 keyword, frequency 컬럼을 가진 데이터프레임이라고 가정
# frequency 기준 내림차순 정렬 후 상위 500개만 추출
top_700 = filtered_df.sort_values(by='frequency', ascending=False).head(700)

# CSV 파일로 저장 (utf-8-sig로 한글 깨짐 방지)
top_700.to_csv('top_700_keywords_일반.csv', index=False, encoding='utf-8-sig')
